In [ ]:
import subprocess
subprocess.check_call(["pip", "install", "selenium"])
subprocess.check_call(["pip", "install", "webdriver-manager"])
subprocess.check_call(["pip", "install", "requests"])

In [88]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
import pandas as pd
import requests

In [64]:
def abrir_navegador():
    """
    Abre o navegador e acessa o site do IBGE.

    :return: Navegador aberto.
    """
    opcoes = Options()
    opcoes.add_argument('--headless') # Rodar sem abrir o navegador
    # opcoes.add_argument('--auto-open-devtools-for-tabs') # Abrir o console do navegador
    # opcoes.add_argument('--start-maximized')
    navegador = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=opcoes)
    return navegador

def fechar_navegador(navegador):
    """
    Fecha o navegador.

    :param navegador: Navegador aberto.
    """
    navegador.quit()

def retornar_tabela():
    """
    Retorna a tabela com os dados de população por município do estado de São Paulo.

    :return: Tabela com os dados do site do IBGE.
    """
    elemento = (By.XPATH, '//*[@id="municipios"]')
    tabela = WebDriverWait(navegador, 20).until(EC.presence_of_element_located(elemento))
    return tabela

def acessar_site(url):
    """
    Acessa o site pela URL.

    :param navegador: Navegador aberto.
    :param url: URL do site do IBGE.
    """
    navegador.get(url)
    tabela = retornar_tabela()
    return tabela


In [80]:
def dados_populacao():
    """
    Retorna um data frame com os dados de população por município do estado de São Paulo.

    :return: Data frame com os dados de população por município do estado de São Paulo.
    """
    tabela = acessar_site('https://cidades.ibge.gov.br/brasil/sintese/sp?indicadores=96385')
    df = pd.read_html(tabela.get_attribute("outerHTML"), flavor="lxml")[0]
    df = df.drop(columns=['Gentílico'])
    df = df.rename(columns={'Municípios': 'cidade', 'População no último censo': 'populacao_censo'})
    df['populacao_censo'] = df['populacao_censo'].str.replace(' pessoas', '')
    return df

def dados_densidade():
    """
    Retorna um data frame com os dados de densidade populacional por município do estado de São Paulo.

    :return: Data frame com os dados de população por município do estado de São Paulo.
    """
    tabela = acessar_site('https://cidades.ibge.gov.br/brasil/sintese/sp?indicadores=96386')
    df = pd.read_html(tabela.get_attribute("outerHTML"), flavor="lxml")[0]
    df.drop(columns=['Gentílico'], inplace=True)
    df.rename(columns={'Municípios': 'cidade', 'Densidade demográfica': 'densidade_demografica'}, inplace=True)
    df['densidade_demografica'] = df['densidade_demografica'].str.replace(' habitante por quilômetro quadrado', '').str.replace(',', '.')
    return df

In [81]:
navegador = abrir_navegador()
df_populacao = dados_populacao()
df_densidade = dados_densidade()
fechar_navegador(navegador)



# unir os dois data frames por cidade
df = pd.merge(df_populacao, df_densidade, on='cidade')
df.head()

C:\Users\ander\AppData\Local\Temp\ipykernel_7720\3767915898.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(tabela.get_attribute("outerHTML"), flavor="lxml")[0]
C:\Users\ander\AppData\Local\Temp\ipykernel_7720\3767915898.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(tabela.get_attribute("outerHTML"), flavor="lxml")[0]


,cidade,populacao_censo,densidade_demografica
0,Adamantina,34687,84.19
1,Adolfo,4351,20.62
2,Aguaí,32072,67.58
3,Águas da Prata,7369,51.65
4,Águas de Lindóia,17930,298.21


In [98]:
url = "https://servicodados.ibge.gov.br/api/v1/localidades/estados/SP/municipios/"

try:
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        df = pd.json_normalize(data)
        df = df[['nome', 'microrregiao.nome', 'microrregiao.mesorregiao.nome', 'regiao-imediata.nome', 'regiao-imediata.regiao-intermediaria.nome']]
        df.rename(columns={
            'nome': 'cidade',
            'microrregiao.nome': 'microrregiao',
            'microrregiao.mesorregiao.nome': 'mesorregiao',
            'regiao-imediata.nome': 'regiao_imediata',
            'regiao-imediata.regiao-intermediaria.nome': 'regiao_intermediaria'}, inplace=True)
    else:
        print(f"A solicitação falhou com status {response.status_code}")
except Exception as e:
    print(f"Ocorreu um erro durante a solicitação: {e}")

df.head()
# incompleto

,cidade,microrregiao,mesorregiao,regiao_imediata,regiao_intermediaria
0,Adamantina,Adamantina,Presidente Prudente,Adamantina - Lucélia,Presidente Prudente
1,Adolfo,São José do Rio Preto,São José do Rio Preto,São José do Rio Preto,São José do Rio Preto
2,Aguaí,Pirassununga,Campinas,São João da Boa Vista,Campinas
3,Águas da Prata,São João da Boa Vista,Campinas,São João da Boa Vista,Campinas
4,Águas de Lindóia,Amparo,Campinas,Amparo,Campinas
